Please see "README_Reproducibility_2024_CDC_Topic_Model_Code_Workbook.ipynb" for an explanation of some differences you will see when you run this notebook in a new environment.

In [1]:
# Standard Libraries and Others
import sys
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import textwrap
import time
import requests
import json
import datetime

In [2]:
import pandas as pd
from bertopic import BERTopic
from transformers.pipelines import pipeline
from transformers import AutoTokenizer, AutoModel
from bertopic.vectorizers import ClassTfidfTransformer

In [3]:
print(os.getcwd())

/home/jovyan/work/Science Clips Topic Model


# Import Data

In [4]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Here, we import Science Clips. 

Science Clips is available as an Excel download from: https://www.cdc.gov/library/sciclips/download/.

Because we used a specific version of Science Clips (accessed 5/3/2024), your results may vary if you use a fresh copy of Science Clips; to best replicate our results, limit the "Date" field of your download to 5/3/2024 or earlier. 

Our copy has a large file size that isn't stored in this GitHub repo. If you would like a copy of the specific version of Science Clips we used, please contact us.

In [5]:
df_clips = pd.read_excel('ScienceClips_accessed20240503.xlsx')

In [7]:
# I will limit my dataset to publications from 2014 to 2023
df_clips = df_clips[df_clips['Year'] <= 2023]
df_clips = df_clips[df_clips['Year'] >= 2014]

In [8]:
df_clips.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34104 entries, 180 to 37160
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Authors                            34104 non-null  object        
 1   CDC Authors                        34104 non-null  object        
 2   Title                              34104 non-null  object        
 3   Abstract                           34104 non-null  object        
 4   Year                               34104 non-null  int64         
 5   Journal                            34104 non-null  object        
 6   Volume                             33508 non-null  object        
 7   Issue                              29395 non-null  object        
 8   Pages                              31925 non-null  object        
 9   PMID                               32925 non-null  float64       
 10  PMCID                              27

In [9]:
docs = list(df_clips['Title'] + ' ' + df_clips['Abstract'])

In [10]:
type(docs)

list

In [28]:
len(docs)

34104

In [37]:
docs[0:5]

['Enhancing acute flaccid paralysis surveillance system towards polio eradication: reverse cold chain monitoring in Nigeria, 2017 to 2019 INTRODUCTION: Highly sensitive acute flaccid paralysis (AFP) surveillance is critical for detection of poliovirus circulation and documentation for polio-free certification. The reverse cold chain (RCC) is a system designed to maintain stool specimens in appropriate temperature for effective detection of poliovirus in the laboratory. We monitored the RCC of AFP surveillance in Nigeria to determine its effectiveness in maintaining viability of enterovirus. METHODS: A descriptive cross-sectional study was conducted from November 2017 to December 2019. We included AFP cases from 151 Local Government Areas and monitored RCC of paired stool specimens from collection to arrival at laboratories. The national guideline recommends RCC temperature of +2 to +8°C and a non-polio enterovirus (NPENT) detection rate of ≥10%. We analyzed data with Epi Info 7, and pr

# 1. Embedding

In [12]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model = SentenceTransformer("allenai-specter")
embeddings = embedding_model.encode(docs, show_progress_bar=True)

Batches:   0%|          | 0/1066 [00:00<?, ?it/s]

In [13]:
embeddings.shape

(34104, 768)

In [29]:
embeddings[0,:]

array([ 5.61787665e-01, -8.40109140e-02,  9.71067399e-02,  1.05580831e+00,
       -3.75183970e-01,  9.47094023e-01,  2.91449666e-01,  9.34312940e-01,
       -9.49569792e-02,  5.81512570e-01, -7.35049993e-02,  1.66970894e-01,
       -2.73748517e-01,  8.17897677e-01, -4.63324487e-01, -9.50737670e-02,
       -1.62628531e+00,  4.71217096e-01, -3.01699638e-02, -3.43132228e-01,
        2.50212457e-02, -2.57567316e-01, -1.20233893e+00,  2.05016756e+00,
       -1.05179989e+00, -2.39362344e-01,  1.54852605e+00, -3.35096382e-02,
       -4.52603340e-01,  1.24597237e-01, -3.88383446e-03,  5.49504273e-02,
       -5.21303773e-01,  4.07696635e-01, -4.51892227e-01,  2.57099979e-02,
       -3.77694637e-01,  4.08637494e-01, -7.74119973e-01,  1.03649783e+00,
       -9.41887498e-01,  2.87357032e-01, -2.94630438e-01,  1.82351023e-01,
        1.21745035e-01, -3.77612442e-01, -2.73459285e-01, -4.07523185e-01,
       -9.95247886e-02,  1.06890464e+00,  1.86224394e-02,  1.32145214e+00,
        7.81384349e-01, -

# 2. Dimensionality Reduction (UMAP)

In [14]:
from umap import UMAP
umap_model = UMAP(n_neighbors=25, n_components=50, min_dist=0.0, metric='cosine', random_state=47)
umap_embeddings = umap_model.fit_transform(embeddings)

In [15]:
umap_embeddings.shape

(34104, 50)

# 3. Clustering (HDBSCAN)

In [18]:
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=175,
                        min_samples=10,
                        metric='euclidean', 
                        cluster_selection_method='eom', 
                        prediction_data=True,
                       gen_min_span_tree = True)
hdbscan_embeddings = hdbscan_model.fit(umap_embeddings)

# Optimization Parameters:
## Coverage:

In [19]:
coverage = (len(hdbscan_embeddings.labels_) - np.sum(hdbscan_embeddings.labels_ == -1))/len(hdbscan_embeddings.labels_)
print(coverage)

0.7784424114473375


## Validity:

In [20]:
validity = hdbscan_embeddings.relative_validity_
print(validity)

0.5007782122861829


## Evenness:

In [21]:
_,counts = np.unique(hdbscan_embeddings.labels_,return_counts = True)
print(counts)

# Old evenness implementation
#cluster_evenness = np.std(counts[1:])
#print(cluster_evenness)

[7556  738  560  307  587  282  275 2931  184  494  379  903  214  345
  559  669  868  204  577  351 3828  175  400  874  522  240  293  279
  525  365  730  859  472  293  181  538  433  353  722  177  569  230
  484  345  603  327  304]
637.6625660991547


In [22]:
evenness = 1 - ((np.max(counts[1:]) - np.min(counts[1:]))/(len(hdbscan_embeddings.labels_)-2))
print(np.max(counts[1:]))
print(np.min(counts[1:]))
print(evenness)

3828
175
0.8928801829804703


In [23]:
len(counts[1:])

46

## Overall Optimization Score:

In [24]:
overall = (coverage/3)+(validity/3)+(evenness/3)
print(overall)

0.724033602237997


# Optimization Implementation:

In [25]:
# Huge grid search implementation adapted by Joy Ortega from Mueller and Guido
# Last modified 2024-05-10

import hdbscan

# I want to know both the best score and best evenness. 
# Coverage will contribute to overall score, but no need to track it
# To initialize these variables, uncomment.
best_score = -1
best_evenness = -1
best_overall = -1

#for n_neighbors in [100,1000,3000,7500]:
for n_neighbors in [15,25,50]:
    for n_components in [25,50]:
        for min_dist in [0,0.25,0.5,0.75,1]:
            # For each combination of parameters of umap
            umap_embeddings = UMAP(n_neighbors=n_neighbors,
                                   n_components=n_components,
                                   min_dist=min_dist,
                                   metric='cosine',
                                   random_state=47).fit_transform(embeddings)
            for min_cluster_size in [25,50,75,100,125,150,175,200,225,250]:
                for min_samples in [10,20,30,40,50,60,70,80,90,100,110,120]:
                    
                    # For each combination of parameters in hdbscan
                    hdb = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size,
                                          min_samples=min_samples,
                                          cluster_selection_method='eom',
                                          metric='euclidean',
                                          gen_min_span_tree=True).fit(umap_embeddings)
                    coverage = (len(hdb.labels_) - np.sum(hdb.labels_ == -1))/len(hdb.labels_)
                    score = hdb.relative_validity_
                    _,counts = np.unique(hdb.labels_,return_counts = True)
                    evenness = 1 - ((np.max(counts[1:]) - np.min(counts[1:]))/(len(hdb.labels_)-2))
                    #evenness = 1 - (np.std(counts[1:])/((len(hdb.labels_)/2)-1))
                    overall = (1/3)*coverage + (1/3)*score + (1/3)*evenness

                    if score > best_score:
                        best_score = score
                        best_score_params = {'n_neighbors': n_neighbors,
                                             'n_components': n_components,
                                             'min_dist': min_dist,
                                             'min_cluster_size': min_cluster_size, 
                                             'min_samples':  min_samples}
                        
                    if evenness > best_evenness:
                        best_evenness = evenness
                        best_evenness_params = {'n_neighbors': n_neighbors,
                                             'n_components': n_components,
                                             'min_dist': min_dist,
                                             'min_cluster_size': min_cluster_size, 
                                             'min_samples':  min_samples}

                    if overall > best_overall:
                        best_overall = overall
                        best_overall_params = {'n_neighbors': n_neighbors,
                                             'n_components': n_components,
                                             'min_dist': min_dist,
                                             'min_cluster_size': min_cluster_size, 
                                             'min_samples':  min_samples}


print("Best DBCV score: {:.3f}".format(best_score))
print("Best DBCV parameters: {}".format(best_score_params))
print("Best evenness: {:.3f}".format(best_evenness))
print("Best evenness: {}".format(best_evenness_params))
print("Best overall score: {:.3f}".format(best_overall))
print("Best overall parameters: {}".format(best_overall_params))

Best DBCV score: 0.739
Best DBCV parameters: {'n_neighbors': 15, 'n_components': 25, 'min_dist': 0.25, 'min_cluster_size': 25, 'min_samples': 40}
Best evenness: 0.988
Best evenness: {'n_neighbors': 25, 'n_components': 25, 'min_dist': 0.75, 'min_cluster_size': 25, 'min_samples': 10}
Best overall score: 0.724
Best overall parameters: {'n_neighbors': 25, 'n_components': 50, 'min_dist': 0, 'min_cluster_size': 175, 'min_samples': 10}


# Topic Model

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))

from bertopic.representation import KeyBERTInspired
# Fine-tune your topic representations
representation_model = KeyBERTInspired()

#from bertopic import BERTopic
#from bertopic.dimensionality import BaseDimensionalityReduction

# Fit BERTopic without actually performing any dimensionality reduction
#empty_dimensionality_model = BaseDimensionalityReduction()

topic_model = BERTopic(embedding_model=embedding_model,
                       umap_model=umap_model,
                      hdbscan_model=hdbscan_model,
                      vectorizer_model=vectorizer_model,
                      representation_model=representation_model)
topics, probs = topic_model.fit_transform(docs,embeddings)

In [54]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,7556,-1_public health_care_surveillance_prevalence,"[public health, care, surveillance, prevalence...","[Abortion surveillance - United States, 2012 P..."
1,0,3828,0_cov infection_vaccine effectiveness_influenz...,"[cov infection, vaccine effectiveness, influen...",[Association Between mRNA Vaccination and COVI...
2,1,2931,1_antiretroviral therapy_chlamydia_sexually tr...,"[antiretroviral therapy, chlamydia, sexually t...",[Low HIV testing rates among U.S. women who re...
3,2,903,2_occupational health_workplace_occupational s...,"[occupational health, workplace, occupational ...","[Reducing risks to women linked to shift work,..."
4,3,874,3_respirable dust_aerosols_sampling_respirable,"[respirable dust, aerosols, sampling, respirab...",[Design and experimental evaluation of a flood...
5,4,868,4_phthalates_urinary concentrations_phthalate_...,"[phthalates, urinary concentrations, phthalate...",[Phthalates and thyroid function in preschool ...
6,5,859,5_malaria cases_malaria transmission_malaria c...,"[malaria cases, malaria transmission, malaria ...","[Malaria surveillance - United States, 2015 PR..."
7,6,738,6_resistant tuberculosis_tuberculosis infectio...,"[resistant tuberculosis, tuberculosis infectio...",[Prospects for tuberculosis elimination in the...
8,7,730,7_diabetes prevention_adults diabetes_type dia...,"[diabetes prevention, adults diabetes, type di...",[Association of type 1 diabetes vs type 2 diab...
9,8,722,8_dengue virus_leptospirosis_leptospira_chikun...,"[dengue virus, leptospirosis, leptospira, chik...","[Epidemiology of dengue, chikungunya, and Zika..."


In [166]:
topic_model.get_topic_info().to_csv('topics_fullSC_v5_bestoverall_SPECTER.csv')

In [167]:
topic_model.get_document_info(docs).to_csv('topics_for_docs_fullSC_v5_bestoverall_SPECTER.csv')

In [171]:
topic_model.save("bestoverall_SPECTER_20240828", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

In [33]:
df_clips_reindex = df_clips.reset_index(drop=True)

In [34]:
df_clips_reindex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34104 entries, 0 to 34103
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Authors                            34104 non-null  object        
 1   CDC Authors                        34104 non-null  object        
 2   Title                              34104 non-null  object        
 3   Abstract                           34104 non-null  object        
 4   Year                               34104 non-null  int64         
 5   Journal                            34104 non-null  object        
 6   Volume                             33508 non-null  object        
 7   Issue                              29395 non-null  object        
 8   Pages                              31925 non-null  object        
 9   PMID                               32925 non-null  float64       
 10  PMCID                             

In [59]:
df_docs = pd.concat([df_clips_reindex,pd.DataFrame(topic_model.get_document_info(docs))],axis=1)

In [56]:
df_docs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34104 entries, 0 to 34103
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Authors                            34104 non-null  object        
 1   CDC Authors                        34104 non-null  object        
 2   Title                              34104 non-null  object        
 3   Abstract                           34104 non-null  object        
 4   Year                               34104 non-null  int64         
 5   Journal                            34104 non-null  object        
 6   Volume                             33508 non-null  object        
 7   Issue                              29395 non-null  object        
 8   Pages                              31925 non-null  object        
 9   PMID                               32925 non-null  float64       
 10  PMCID                             

In [176]:
df_docs.to_excel('ScienceClips_with_Topics_bestoverall_nneigh25_2014-2023_forImpact.xlsx')

# Get 2D Embeddings:

In [ ]:
# The only difference between this model and the previous one is that this model uses only 2 components.
umap_model = UMAP(n_neighbors=25, n_components=2, min_dist=0.0, metric='cosine', random_state=47)
umap_embeddings = umap_model.fit_transform(embeddings)

In [29]:
df_embeddings = pd.DataFrame(umap_embeddings,columns=['UMAP 1', 'UMAP 2'])

In [57]:
df_embeddings.head()

,UMAP 1,UMAP 2
0,2.637221,5.079272
1,9.508604,0.321237
2,11.176594,-1.294658
3,2.078050,0.166428
4,3.117112,-2.922476


In [60]:
df_embeddings = pd.concat([pd.DataFrame(topic_model.get_document_info(docs)),df_embeddings],axis=1)

In [61]:
df_embeddings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34104 entries, 0 to 34103
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Document                 34104 non-null  object 
 1   Topic                    34104 non-null  int64  
 2   Name                     34104 non-null  object 
 3   Representation           34104 non-null  object 
 4   Representative_Docs      34104 non-null  object 
 5   Top_n_words              34104 non-null  object 
 6   Probability              34104 non-null  float64
 7   Representative_document  34104 non-null  bool   
 8   UMAP 1                   34104 non-null  float32
 9   UMAP 2                   34104 non-null  float32
dtypes: bool(1), float32(2), float64(1), int64(1), object(5)
memory usage: 2.1+ MB


In [62]:
df_embeddings.to_excel('Embeddings_Visualizations_BestOverall.xlsx')